# Sistema de Compressão

$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^{-3} m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 MPa \\
P{out} & = & 5.0 MPa \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(1.5\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

#### Importações

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.optimize import fsolve
import casadi as ca
import plotly.graph_objects as go
import optuna
from plotly.subplots import make_subplots
import time
import matplotlib.pyplot as plt

2024-10-24 22:38:34.964205: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 22:38:34.967559: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 22:38:34.978348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 22:38:34.995430: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 22:38:35.000212: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 22:38:35.012877: I tensorflow/core/platform/cpu_feature_gu

#### Constantes e Variáveis Auxiliares

In [76]:
np.random.seed(42)
print(np.random.seed)

# Constantes
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 4.5
P_out = 5
C = 479

timestep = 3 # Passos no passado para prever o próximo
epochs = 5000
nData = 1000 
nAlphas = 5 # Número de vezes que o Alfa irá mudar, considere o treino e os testes.
alphas = np.random.uniform(0.35,0.65, nAlphas+1) # Abertura da válvula
perturb = 1e-3
tempo = 100
dt = 0.1 # Tempo amostral

# Variáveis auxiliares
interval = [np.linspace(i * tempo, (i + 1) * tempo, nData) for i in range(nAlphas)]
interval_test = [np.linspace(i * 60, (i + 1) * 60, 600) for i in range(nAlphas)]
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []

<built-in function seed>


### Solução Numérica

##### Cálculo da Solução

In [56]:


# Condições Iniciais
def fun(variables, A1, Lc, kv, P1, P_out, C) :
    (x,y) = variables
    eqn_1 = (A1/Lc)* ((1.5 * P1) - y)*1e3
    eqn_2 = (C**2)/2 * (x - alphas[0] * kv * np.sqrt(y*1000 - P_out*1000))
    return [eqn_1, eqn_2]

result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C)) 

init_m = result[0] 
init_p = result[1]

# Variáveis CasADi
x = ca.MX.sym('x', 2)
alpha = ca.MX.sym('alpha', 1)

# Solução Numérica
tm1 = time.time()
for i in range(nAlphas):
    alpha_values.append(alphas[i] + np.random.normal(0,perturb,nData))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e3, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

    F = ca.integrator('F','idas', ode, interval[0][0], dt)

    for j in range(nData):
        sol = F(x0 = [init_m, init_p], p = alpha_values[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrate.append(aux1)
        PlenumPressure.append(aux2)
        init_m = aux1[-1]
        init_p = aux2[-1]
        RNN_train.append([aux1[0], aux2[0], alpha_values[i][j]])
        RNN_trainFut.append([aux1[0], aux2[0], alpha_values[i][j]])
    
tm2 = time.time()

massFlowrate = np.reshape(massFlowrate, [nAlphas,nData])
PlenumPressure = np.reshape(PlenumPressure, [nAlphas,nData])

print(tm2-tm1)
print(RNN_train)


3.851292610168457
[[7.351447130061303, 6.738811494495828, 0.4639412484697161], [7.352084177557419, 6.745214893828777, 0.46312947038336166], [7.351511918157013, 6.754301378852157, 0.46189256126827377], [7.3519328819603516, 6.746839030471486, 0.4629045956977947], [7.351374817106706, 6.75418994105393, 0.4618986179613963], [7.350848539425641, 6.753956347001726, 0.4618963059006385], [7.351024419553049, 6.748677706600474, 0.4626039979257748], [7.3491293283453265, 6.764176323286021, 0.46044875540955094], [7.347533337543864, 6.761968275363956, 0.4606371178216957], [7.347145397866058, 6.752922413854528, 0.46179974812496777], [7.34635053935904, 6.7559678209887135, 0.46134920453387435], [7.346863088282142, 6.746152707981595, 0.462676282986804], [7.346185695100297, 6.75509149905481, 0.46145401157868754], [7.345066736977126, 6.758397958551045, 0.4609497319528735], [7.3467428111535265, 6.73743729108393, 0.4638276844231303], [7.346720525775621, 6.750168068723585, 0.4621362593537222], [7.3469770452756

##### Gráfico do Modelo

In [57]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Vazão vs Tempo", "Pressão vs Tempo", "Alpha vs Tempo"))

for i in range(0, nAlphas):
    # Vazão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Vazão', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    # Pressão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Pressão', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    # Alphas
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alphas', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 1, col = 3)

# Atualiza layout
fig.update_layout(
    xaxis_title='Tempo',
    grid=dict(rows=1, columns=3),
    template='plotly',
    showlegend=False
)

# Mostra a figura
fig.show()


### Rede Neural

##### Dados de Treino

In [58]:
newData = int((nData*nAlphas)/(timestep + 1))
RNN_train = np.array(RNN_train)


X_train = []
y_train = []

'''
for i in range(len(RNN_train) - 3):
    X_train.append(RNN_train[i:i + 3])          # Os 3 timesteps
    y_train.append(RNN_train[i + 1:i + 3 + 1, :2])
'''

for i in range(len(RNN_train) - timestep):
    X_train.append(RNN_train[i:i + timestep])  
    if i + timestep < len(RNN_train):           
        y_train.append(RNN_train[i + timestep, :2])  

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

y_train = tf.expand_dims(y_train, axis=1)


##### Rede

In [101]:


class NormLayer(keras.Layer):
    def call(self, inputs):
        batch_min = tf.reduce_min(inputs, axis=[1, 2], keepdims=True)
        batch_max = tf.reduce_max(inputs, axis=[1, 2], keepdims=True)
        return 2 * (inputs - batch_min) / (batch_max - batch_min) - 1, batch_min, batch_max

class DenormLayer(keras.Layer):
    def call(self, inputs, batch_min, batch_max):
        return ((inputs + 1) / 2) * (batch_max - batch_min) + batch_min

class MyModel(tf.keras.Model):
    def __init__(self, num_layers, units):
        super(MyModel, self).__init__()
        #self.normalize_layer = NormLayer()
        #self.denormalize_layer = DenormLayer()
        self.rnn_layers = []
        self.num_layers = num_layers
        self.units = units
        self.inputs = None

    def build(self):
        self.input_layer = keras.layers.InputLayer(input_shape=(3,3))
        self.rnn_layers.append(keras.layers.Bidirectional(keras.layers.SimpleRNN(self.units, return_sequences = True, activation='tanh', kernel_initializer='glorot_uniform')))
        for _ in range(self.num_layers):
            self.rnn_layers.append(keras.layers.SimpleRNN(self.units, return_sequences = True, activation='tanh', kernel_initializer='glorot_uniform'))
        self.rnn_layers.append(keras.layers.SimpleRNN(self.units, return_sequences = False, activation='tanh', kernel_initializer='glorot_uniform'))
        self.dense = keras.layers.Dense(2, kernel_initializer='glorot_uniform')

    def call(self, inputs):
        self.inputs = inputs
        #normalized_inputs, batch_min, batch_max = self.normalize_layer(inputs)
        rnn_output = self.inputs
        for rnn_layer in self.rnn_layers:
            rnn_output = rnn_layer(rnn_output)
        dense_output = self.dense(rnn_output)
        return dense_output #self.denormalize_layer(dense_output, batch_min, batch_max)

@tf.function
def lossCustom(y_true, y_pred,inputs):
    #m_t = (11* y_pred[:, 0] - 18 *inputs[:, -2, 0] + 9 * inputs[:, -3, 0] - 2 * inputs[:, 0, 0])/6*dt
    #p_t = (11* y_pred[:, 1] - 18 *inputs[:, -2, 1] + 9 * inputs[:, -2, 1] - 2 * inputs[:, -2, 1])/6*dt
    #fLoss_mass = tf.reduce_mean(tf.square(m_t - (A1/Lc)*((1.5 * P1) - y_pred[:, 1]) * 1e3))
    #fLoss_pres = tf.reduce_mean(tf.square(p_t - (C**2)/2 * (y_pred[:, 0] - inputs[:, -1, -1]* kv * tf.sqrt((tf.abs(y_pred[:, 1] * 1000 - P_out * 1000))))))
    data_loss = tf.reduce_mean(tf.square(y_true[:, 0, 0] - y_pred[:, 0])) + 1e3*tf.reduce_mean(tf.square(y_true[:, 0, 1] - y_pred[:, 1]))
    
    return data_loss #+ 1e-10*(fLoss_mass + fLoss_pres)

def create_model(lr, num_layers, units):
    model = MyModel(num_layers, units)

    def lossCustomWrapper(y_true, y_pred):
        return lossCustom(y_true, y_pred, model.inputs)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr, clipvalue=1.0), loss=lossCustomWrapper)
    return model    

early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',  
    patience=epochs//10,     
    restore_best_weights=True 
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',  
    factor=0.5,          
    patience=100,          
    min_lr=1e-8          
)

##### Treinamento

In [ ]:
def objective(trial):
    lr = trial.suggest_float('lr', 1e-6, 1e-3, log=True) 
    units = trial.suggest_int('units', 32, 64)
    batch_size = trial.suggest_int('batch_size', 16, 64) 
    num_layers = trial.suggest_int('num_layers', 0, 5)  

    model = create_model(lr, num_layers, units)

    model.fit(X_train, y_train, epochs=250, batch_size=batch_size, verbose=0)

    # Avaliação
    loss = model.evaluate(X_train, y_train, verbose=0)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)  # Ajuste o número de tentativas conforme necessário

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros: ", study.best_params)

In [97]:
# model = create_model(study.best_params['lr'], study.best_params['num_layers'], study.best_params['units'])
model = create_model(1e-4, 2, 60)

history = model.fit(X_train, y_train, epochs=1000, verbose=1, batch_size= 128) #batch_size=study.best_params['batch_size']

Epoch 1/1000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning:

Argument `input_shape` is deprecated. Use `shape` instead.



40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 43589.7109
Epoch 2/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 27589.3672
Epoch 3/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10845.2568
Epoch 4/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1168.3940
Epoch 5/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 46.3499
Epoch 6/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 40.2613
Epoch 7/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 34.3813
Epoch 8/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 31.4470
Epoch 9/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 25.8188
Epoch 10/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 22.3027
Epoch 11/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 19.7150
Epoch 12/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 16.9309
Epoch 13/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 13.9740
Epoch 14/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 12.5418
Epoch 15/1000
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8

##### Dados de teste

In [98]:
massFlowrateTeste = []
PlenumPressureTeste = []
RNN_test = []
x_test = []
alpha_valuesTeste = []
alphasTeste = np.random.uniform(0.35,0.65, nAlphas) # Abertura da válvula
aux1 = []
aux2 = []

tm1 = time.time()
for i in range(0,nAlphas):
    alpha_valuesTeste.append(alphasTeste[i] + np.random.normal(0,perturb,nData))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e3, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }
    
    for j in range(600):
        F = ca.integrator('F','idas', ode, interval[0][0], dt)
        sol = F(x0 = [init_m, init_p], p = alpha_valuesTeste[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrateTeste.append(aux1)
        PlenumPressureTeste.append(aux2)
        init_m = aux1[-1]
        init_p = aux2[-1]
        RNN_test.append([aux1[0], aux2[0], alpha_valuesTeste[i][j]])

tm2 = time.time()
print(tm2-tm1)

RNN_test = np.array(RNN_test)

for i in range(len(RNN_test) - 3):
    x_test.append(RNN_test[i:i + 3]) 

x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
massFlowrateTeste = np.array(massFlowrateTeste)
PlenumPressureTeste = np.array(PlenumPressureTeste)
massFlowrateTeste = np.reshape(massFlowrateTeste, [nAlphas, 600])
PlenumPressureTeste = np.reshape(PlenumPressureTeste, [nAlphas, 600])

4.071479797363281


##### Gráfico Rede Neural

In [99]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

# Loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)), y=history.history['loss'], mode='lines', name='Mass Loss', line=dict(dash='solid')),
                  row=1, col=1)

tm1 = time.time()
# for i in range(newData):
    # inputs = np.array([temp1,temp2])
    # prediction = model.predict(inputs)
    # temp1 = x_test[:,1,:]
    # temp2 = prediction[0,:,:]
    # mass.append(prediction[:,0,:])
    # pressure.append(prediction[:,0,:])

prediction = model.predict(x_test)
print(prediction.shape)
mass = prediction[:, 0]
pressure = prediction[:, 1]

tm2 = time.time()
print(tm2-tm1)

interval2 = [np.linspace(0, tempo*nAlphas, newData) for i in range(nAlphas)]

interval3 = np.linspace(0, 300, 2997)

fig.add_trace(go.Scatter(x=interval3,y=np.squeeze(mass),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
fig.add_trace(go.Scatter(x=interval3,y=np.squeeze(pressure),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)


for i in range(nAlphas):
    '''
    for j in range(0, len(interval2[i]), nData):
        end = j + nData
        if end > len(interval2[i]):
            end = len(interval2[i])
        
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(mass[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(pressure[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)
    '''
    # Modelo
    fig.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig.show()


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
(2997, 2)
1.1633129119873047


In [100]:
massFlowrate100 = [x_test[0, 0, 0], x_test[0, 1, 0], x_test[0, 2, 0]]
PlenumPressure100 = [x_test[0, 0, 1], x_test[0, 1, 1], x_test[0, 2, 1]]

for i in range (0, 2997):
    inputs = np.array([[
    [massFlowrate100[-3], PlenumPressure100[-3], x_test[i, 0, 2].numpy()],
    [massFlowrate100[-2], PlenumPressure100[-2], x_test[i, 1, 2].numpy()],
    [massFlowrate100[-1], PlenumPressure100[-1], x_test[i, 2, 2].numpy()]
    ]])
    prediction100 = model.predict(inputs)
    massFlowrate100.append(prediction100[0, 0])
    print(prediction100[0, 0])
    PlenumPressure100.append(prediction100[0, 1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
7.140144
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
7.0281324
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.8817954
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6.6934967
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6.6065574
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.520691
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.4446573
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6.3970895
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.3564157
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.3250184
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.3039784
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.288375
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.278225
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.2729354
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6.270922
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6.271847
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6.2752237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
6.2803006
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.286837
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.2944865
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
6.3027425

KeyboardInterrupt: 

In [95]:
fig3 = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

fig3.add_trace(go.Scatter(x=interval3,y=np.squeeze(massFlowrate100),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
fig3.add_trace(go.Scatter(x=interval3,y=np.squeeze(PlenumPressure100),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)



for i in range(nAlphas):
    # Modelo
    fig3.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig3.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig3.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig3.show()